In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Here write the path where you cloned the repository + ML_2019_EPFL/projects/project2/project_road_segmentation/

---



In [0]:
%cd drive/My\ Drive/ML_2019_EPFL/projects/project2/project_road_segmentation/

/content/drive/My Drive/ML_2019_EPFL/projects/project2/project_road_segmentation


In [0]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D, LeakyReLU

In [0]:
import numpy as np
import scipy as scipy
import matplotlib.pyplot as plt
import os
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as keras
from helpers import *

In [0]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D, LeakyReLU

In [0]:
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot

import cv2
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


In [0]:
# Load a set of images
root_dir = "data/training/"

# Select the directory for the images and load them
image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = len(files) 

print("Loading " + str(n) + " images")
imgs = np.asarray([load_image(image_dir + files[i]) for i in range(n)])

# Select the directory for groundtruth images and load them
gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " groundtruth images")
gt_imgs = np.asarray([load_image(gt_dir + files[i]) for i in range(n)])


Loading 100 images
Loading 100 groundtruth images


# Generating mini-batch and running data augmentation¶

In [0]:
def pad_matrix(mat,h_pad,w_pad,val=0):
  h_pad = int(h_pad)
  w_pad = int(w_pad)
  if len(mat.shape)==3:
    padded_mat = np.pad(mat,((h_pad,h_pad),(w_pad,w_pad),(0,0)),mode='constant',constant_values=((val,val),(val,val),(0,0)))
  elif len(mat.shape)==2:
    padded_mat = np.pad(mat,((h_pad,h_pad),(w_pad,w_pad)),mode='constant',constant_values=((val,val),(val,val)))
  else:
    raise ValueError("This method can only handle 2d or 3d arrays")
  return padded_mat

def imag_rotation(X,Y,number_rotations = 8): 
  
  w = X.shape[1]
  w_2 = w//2 # half of the width
  padding = 82
  padding2 = 24
  Xrs = X
  Yrs = Y
  
  ### Add padding2
  Xrs = pad_matrix(Xrs,padding2,padding2) 
  Yrs = pad_matrix(Yrs,padding2,padding2) 
    ###

  Xrs = np.expand_dims(Xrs, 0)
  Yrs = np.expand_dims(Yrs, 0)

  thetas = np.random.randint(0,high = 360,size = number_rotations)
  for theta in thetas:
    Xr = pad_matrix(X,padding,padding) # Selected for the specific case of images of (400,400) 
    Yr = pad_matrix(Y,padding,padding) # Selected for the specific case of images of (400,400) 
    Xr = scipy.ndimage.rotate(Xr, theta , reshape = False) 
    Yr = scipy.ndimage.rotate(Yr, theta , reshape = False) 
    theta = theta*np.pi/180
    a = int(w_2/(np.sqrt(2)*np.cos(np.pi/4-np.mod(theta,np.pi/2)))) # width and height of the biggest square inside the rotated square 
    w_p = w_2+padding
    Xr = Xr[w_p-a:w_p+a,w_p-a:w_p+a,:]
    Yr = Yr[w_p-a:w_p+a,w_p-a:w_p+a]

    Xr = cv2.resize(Xr, dsize=(w_2*2, w_2*2), interpolation=cv2.INTER_CUBIC)
    Yr = cv2.resize(Yr, dsize=(w_2*2, w_2*2), interpolation=cv2.INTER_CUBIC)

    if np.random.choice(2) == 1:
      Xr = np.flipud(Xr)
      Yr = np.flipud(Yr)
            
    if np.random.choice(2) == 1:
      Xr = np.fliplr(Xr)
      Yr = np.fliplr(Yr)
    
    ### Add padding2
    Xr = pad_matrix(Xr,padding2,padding2) 
    Yr = pad_matrix(Yr,padding2,padding2) 
    ###

    Xr = np.expand_dims(Xr,0)
    Yr = np.expand_dims(Yr,0)
    Xrs = np.append(Xrs,Xr,axis = 0)
    Yrs = np.append(Yrs,Yr,axis = 0)

  return Xrs, Yrs

def imag_rotation_aug(Xr,Yr,number_rotations = 8):
  
  Xrs,Yrs = imag_rotation(Xr[0],Yr[0])
  for i in range(1,len(Xr)):
    Xrr,Yrr = imag_rotation(Xr[i],Yr[i])
    Xrs = np.append(Xrs,Xrr,axis = 0)
    Yrs = np.append(Yrs,Yrr,axis = 0)

  Xrs_shuf = []
  Yrs_shuf = []
  index_shuf = list(range(len(Xrs)))
  np.random.shuffle(index_shuf)
  for i in index_shuf:
    Xrs_shuf.append(Xrs[i])
    Yrs_shuf.append(Yrs[i])

  return Xrs_shuf, Yrs_shuf


In [0]:
X,Y = imag_rotation_aug(imgs,gt_imgs)

In [0]:
X = np.asarray(X)
Y = np.asarray(Y)

In [0]:
print(X.shape)
print(Y.shape)

(900, 448, 448, 3)
(900, 448, 448)


In [0]:
def create_minibatch():
    
    # Fix the seed
    #np.random.seed(1)
    
    # We define the window size of 72, batch size of 100 (empirically chosen)
    # and patch size should correspond to 16
    w_size = 64
    batch_size = 1200
    patch_size = 16
    num_images = 900
    
    while True:
        # Generate one minibatch
        batch_image = np.empty((batch_size, w_size, w_size, 3))
        batch_label = np.empty((batch_size, 2))
        
        for i in range(batch_size):
            
            # Select a random index representing an image
            random_index = np.random.choice(num_images)
            
            # Width of original image
            width = 448
            
            # Sample a random window from the image
            random_sample = np.random.randint(w_size//2, width - w_size//2, 2)
            
            # Create a sub image of size 72x72
            sampled_image = X[random_index][random_sample[0] - w_size // 2 : random_sample[0] + w_size//2,
                                            random_sample[1] - w_size//2 : random_sample[1] + w_size//2]
                
            # Take its corresponding ground-truth image
            correspond_ground_truth = Y[random_index][random_sample[0] - patch_size//2:random_sample[0] + patch_size//2,
                                                      random_sample[1]-patch_size//2:random_sample[1] + patch_size//2]
            
            # Additional flip 
            if np.random.choice(2) == 1:
                sampled_image = np.flipud(sampled_image)
                correspond_ground_truth = np.flipud(correspond_ground_truth)
            if np.random.choice(2) == 1:
                sampled_image = np.fliplr(sampled_image)
                correspond_ground_truth = np.fliplr(correspond_ground_truth)
    


            # We set in the label depending on the threshold of 0.25
            # The label becomes either 0 or 1 by applying to_categorical with parameter 2

            label = to_categorical((np.array([np.mean(correspond_ground_truth)]) > 0.25) * 1,2)
                        
            # We put in the sub image and its corresponding label before yielding it
            batch_image[i] = sampled_image
            batch_label[i] = label

            i+=0

        # Yield the mini_batch to the model
        yield(batch_image, batch_label)

# Creating the class (Same as in cnn_model.py, but provided here for better readability)

In [0]:
class cnn_model:
    
    # Initialize the class
    def __init__(self, shape):
        self.shape = shape
        self.model = self.initialize_cnn_model(shape)
    
    def initialize_cnn_model(self, shape):
        
        # INPUT
        # shape     - Size of the input images
        # OUTPUT
        # model    - Compiled CNN
        
        # Define hyperparamters
        KERNEL3 = (3, 3)
        KERNEL5 = (5, 5)
        
        # Define a model
        model = Sequential()
        
        # Add the layers
        # Selection of the model is described in the report
        # We use padding = 'same' to avoid issues with the matrix sizes
        model.add(Conv2D(64, KERNEL5, input_shape = shape, padding='same'))
        model.add(LeakyReLU(alpha=0.1))
        model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(128, KERNEL3, padding='same'))
        model.add(LeakyReLU(alpha=0.1))
        model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(256, KERNEL3, padding='same'))
        model.add(LeakyReLU(alpha=0.1))
        model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(512, KERNEL3, padding='same'))
        model.add(LeakyReLU(alpha=0.1))
        model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
        model.add(Dropout(0.25))
        
        # Flatten it and use regularizers to avoid overfitting
        # The parameters have been chosen empirically
        model.add(Flatten())
        model.add(Dense(128, kernel_regularizer=l2(0.000001), activity_regularizer=l2(0.000001)))
        model.add(LeakyReLU(alpha=0.1))
        model.add(Dropout(0.5))
        
        # Add output layer
        model.add(Dense(2, kernel_regularizer=l2(0.000001), activity_regularizer=l2(0.000001)))
        model.add(Activation('sigmoid'))
        
        # Compile the model using the binary crossentropy loss and the Adam optimizer for it
        # We used the accuracy as a metric, but F1 score is also a plausible choice
        model.compile(loss='binary_crossentropy',
                      optimizer=Adam(lr=0.001),
                      metrics=['accuracy'])
            
        # Print a summary of the model to see what has been generated
        model.summary()
                      
        return model
    
    def train(self):
        
        # We define the number of epochs and steps per epochs
        EPOCHS = 50
        STEPS_PER_EPOCH = 300
        
        # Early stopping callback after 10 steps
        early_stopping = EarlyStopping(monitor='loss', min_delta=0, patience=10, verbose=1, mode='auto')
        
        # Reduce learning rate on plateau after 4 steps
        lr_callback = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=4, verbose=1, mode='auto')
        
        # Place the callbacks in a list to be used when training
        callbacks = [early_stopping]
        
        # Train the model using the previously defined functions and callbacks
        history = self.model.fit_generator(create_minibatch(), steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS, workers=1, callbacks=callbacks, verbose=1)
        return history

    def classify(self, X):
        # Subdivide the images into blocks with a stride and patch_size of 16
        img_patches = create_patches(X, 16, 16, padding=24)
        
        # Predict
        predictions = self.model.predict(img_patches)
        predictions = (predictions[:,0] < predictions[:,1]) * 1
        
        # Regroup patches into images
        return group_patches(predictions, X.shape[0])
    
    def load(self, filename):
        # Load the model (used for submission)
        self.model = load_model(filename)
    
    def save(self, filename):
        # Save the model (used to then load to submit)
        self.model.save(filename)

In [0]:
# Instantiate the model with the size 72x72, the window size of the images to be fed
model = cnn_model(shape = (64, 64, 3))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 64)        4864      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 64, 64, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
leaky_re_lu_1

In [0]:
# Train the model
history = model.train()

Epoch 1/50
300/300 [==============================] - 327s 1s/step - loss: 0.4598 - acc: 0.7826
Epoch 2/50
300/300 [==============================] - 316s 1s/step - loss: 0.3018 - acc: 0.8632
Epoch 3/50
300/300 [==============================] - 316s 1s/step - loss: 0.2378 - acc: 0.8965
Epoch 4/50
300/300 [==============================] - 315s 1s/step - loss: 0.2077 - acc: 0.9125
Epoch 5/50
300/300 [==============================] - 315s 1s/step - loss: 0.1921 - acc: 0.9199
Epoch 6/50
300/300 [==============================] - 315s 1s/step - loss: 0.1819 - acc: 0.9249
Epoch 7/50
300/300 [==============================] - 318s 1s/step - loss: 0.1716 - acc: 0.9298
Epoch 8/50
300/300 [==============================] - 318s 1s/step - loss: 0.1668 - acc: 0.9322
Epoch 9/50
300/300 [==============================] - 318s 1s/step - loss: 0.1592 - acc: 0.9355
Epoch 10/50
300/300 [==============================] - 318s 1s/step - loss: 0.1543 - acc: 0.9375
Epoch 11/50
300/300 [==================

In [0]:

model.save("AugV10-1.h5")

print(history.history.keys())


In [0]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
#plt.plot(history.history['1r'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
#plt.plot(history.history['1r'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


NameError: ignored

In [0]:

model.load("AugV9-6.h5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
model.classify()

In [0]:
from helpers import *

# from cnn_model import cnn_model

# Instantiate the model
batch_normalization = True
activation = "LeakyReLU"
#model = cnn_model(
#    shape=(72, 72, 3), batch_normalization=batch_normalization, activation=activation
#)

# Load the model
#model.load("AugV8.h5") # cargar tu modelo con los mejores weights

# Print a summary to make sure the correct model is used
model.model.summary()

# We add all test images to an array, used later for generating a submission
image_filenames = []
for i in range(1, 51):
    image_filename = "data/test_set_images/test_" + str(i) + "/test_" + str(i) + ".png"
    image_filenames.append(image_filename)

# Set-up submission filename
submission_filename = "blablabla7.csv"

# Generates the submission
generate_submission(model, submission_filename, *image_filenames)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 72, 72, 64)        4864      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 72, 72, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 36, 36, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 36, 36, 128)       73856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 36, 36, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 18, 18, 128)       0